<a href="https://colab.research.google.com/github/saeu5407/daily_tensorflow/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2seq

seq2seq를 활용해 기계번역을 진행해보도록 하겠습니다.

[원본 텐서플로우 튜토리얼 링크](https://www.tensorflow.org/text/tutorials/nmt_with_attention?hl=ko)


In [1]:
pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 4.6 MB/s 


In [2]:
import numpy as np

import typing
from typing import Any, Tuple

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import tensorflow_text as tf_text

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [3]:
# 모양을 확인하는 데 사용하기 위해 정의해 둔 저수준 API 관련 옵션
use_builtins = True

하단 코드는 모양 검사를 위한 코드입니다.

```
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    if isinstance(names, str):
      names = (names,)

    shape = tf.shape(tensor)
    rank = tf.rank(tensor)

    if rank != len(names):
      raise ValueError(f'Rank mismatch:\n'
                       f'    found {rank}: {shape.numpy()}\n'
                       f'    expected {len(names)}: {names}\n')

    for i, name in enumerate(names):
      if isinstance(name, int):
        old_dim = name
      else:
        old_dim = self.shapes.get(name, None)
      new_dim = shape[i]

      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")
```

## 데이터셋 로드 및 준비

텐서플로우 튜토리얼을 따라 영어-스페인어 데이터 세트를 사용해 데이터셋을 로드합니다.
해당 데이터세트는 다음과 같이 한 쌍으로 구성되어 있습니다.

```
May I borrow this book? ¿Puedo tomar prestado este libro?
```

다운받은 데이터에 대해 다음과 같은 작업을 진행합니다.
1. 각 문장에 토큰 시작과 끝을 추가
2. 특수 문자 제거
3. 단어 -> id, id -> 단어 등 사전 매핑
4. 각 문장을 최대 길이로 채움

### 데이터 다운로드 및 로드

In [4]:
# 데이터 다운로드
import pathlib

path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

2654208/2638744 [==============================] - 0s 0us/step


In [5]:
# 데이터 로드 함수 정의
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines() # 여러 라인으로 구분된 문자열을 한 라인씩 분리하여 리스트 반환
  pairs = [line.split('\t') for line in lines] # 그 리스트를 탭으로 분리하여 영어, 스페인 문장을 나눔

  inp = [inp for targ, inp in pairs]
  targ = [targ for targ, inp in pairs]

  return targ, inp

In [6]:
# 데이터 로드 및 샘플 프린트
targ, inp = load_data(path_to_file)
print("스페인어")
print(inp[-1])
print("영어")
print(targ[-1])

스페인어
Si quieres sonar como un hablante nativo, debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un músico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado.
영어
If you want to sound like a native speaker, you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.


### 데이터셋 생성

`tf.data.Dataset` 을 활용하여 데이터셋을 생성합니다.

In [7]:
BUFFER_SIZE = len(inp)
BATCH_SIZE = 64

# tf.data.Dataset.from_tensor_slices는 알아서 slice를 한다. -> 배치를 위한 전처리
dataset = tf.data.Dataset.from_tensor_slices((inp, targ)).shuffle(BUFFER_SIZE) # shuffle로 랜덤하게 넣는다
# 를 배치에 적용
dataset = dataset.batch(BATCH_SIZE)

In [8]:
# 배치 적용 샘플 테스트
for example_input_batch, example_target_batch in dataset.take(1):
  print(example_input_batch[:5])
  print()
  print(example_target_batch[:5])
  break

tf.Tensor(
[b'\xc2\xbfEn qu\xc3\xa9 camino queda la playa?'
 b'Es un poco m\xc3\xa1s tarde de las once menos cuarto.'
 b'\xc3\x89l puede leer lo suficiente.'
 b'Tom tiene una hermana en Boston.' b'Nos subimos al bus en Shinjuku.'], shape=(5,), dtype=string)

tf.Tensor(
[b'Which way is the beach?' b'It is a little after a quarter to eleven.'
 b'He can read well.' b'Tom has a sister in Boston.'
 b'We got on the bus at Shinjuku.'], shape=(5,), dtype=string)


### 텍스트 전처리

이번 튜토리얼에서는 `tf.saved_model`을 활용하여 모델을 저장하는 것을 목표로 하고 있습니다.

우선 표준화 작업부터 진행합니다.

#### 표준화

악센트를 분할하고, 호환성 문자를 해당 ASCII 문자로 대체하겠습니다.<br>
`tensorflow_text` 패키지에 해당 작업들이 존재합니다.

In [9]:
# 전처리 샘플
example_text = tf.constant('¿Todavía está en casa?')

print(example_text.numpy())
print(tf_text.normalize_utf8(example_text, 'NFKD').numpy())

b'\xc2\xbfTodav\xc3\xada est\xc3\xa1 en casa?'
b'\xc2\xbfTodavi\xcc\x81a esta\xcc\x81 en casa?'


In [10]:
# Standardization Function
def tf_lower_and_split_punct(text):
  # Split accecented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [11]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

¿Todavía está en casa?
[START] ¿ todavia esta en casa ? [END]


#### 텍스트 벡터화

위의 Standardization Function 함수는 `tf.keras.layers.TextVectorization` 안에서 텍스트를 벡터화합니다.

`tf.keras.layers.TextVectorization`는 어휘 추출 및 입력 텍스트를 토큰 시퀀스로 변환하는 레이어입니다.

TextVectorization 층 또는 다른 전처리 층들은 `adapt` 메서드을 가지고 있습니다.
이 메서드는 우선 트레이닝 데이터의 한 에포크를 읽은 후, `Model.fix`와 같이 작업을 수행합니다.
이 `adapt` 메서드는 데이터에 기반하여 레이어를 초기화합니다. 여기에서 어휘를 결정합니다.

In [11]:
max_vocab_size = 5000

input_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

In [11]:
input_text_processor.adapt(inp)

# Here are the first 10 words from the vocabulary:
input_text_processor.get_vocabulary()[:10]

스페인어 TextVectorization 레이어가 영어로 build, .adapt()되는 예시

In [ ]:
output_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

output_text_processor.adapt(targ)
output_text_processor.get_vocabulary()[:10]